# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных:

id — уникальный идентификатор скважины;


f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);


product — объём запасов в скважине (тыс. баррелей).

Условия задачи:

Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.


## Загрузка и подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

In [2]:
geo_0=pd.read_csv('/datasets/geo_data_0.csv') #загрузим данные
geo_1=pd.read_csv('/datasets/geo_data_1.csv')
geo_2=pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
geo_0.info() #изучим данные
geo_1.info()
geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [4]:
geo_0.duplicated().sum() #проверим данные на дубликаты
geo_1.duplicated().sum()
geo_2.duplicated().sum()

0

In [5]:
geo_0.isna().sum() #проверим данные на пропуски
geo_1.isna().sum()
geo_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [6]:
geo_0.describe()  #проверим данные на значимые выбросы
geo_1.describe()
geo_2.describe()

,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


In [7]:
geo_0 = geo_0.drop('id', axis=1)    #удалим столбец id , он нам не понадобится для дальнейшего анализа
geo_1 = geo_1.drop('id', axis=1)
geo_2 = geo_2.drop('id', axis=1)

Пропусков нет , дубликатов нет , типы данных верные, среднее близко к медианному значению , значит значительных выбросов нет.

## Обучение и проверка модели

In [8]:
def region(geo_012): # функция для деления на выбоки, расчет среднего , рмсе
    features = geo_012.drop(['product'],axis = 1) 
    target = geo_012['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size = 0.25, random_state=12345)
    model = LinearRegression(n_jobs=-1).fit(features_train, target_train) #делим на выборки , обучаем модель
    predictions = model.predict(features_valid)
    
    mse = mean_squared_error(target_valid, predictions)
    rmse = mse ** 0.5 #вычисляем rmse

    mean = geo_012['product'].mean()
    mean_pred = predictions.mean()#среднее
    
    target_valid = target_valid.reset_index(drop=True)
    
    return predictions, rmse, mean_pred, target_valid


Регион 0

In [9]:
predictions_0, rmse_0, mean_pred_0, target_valid_0 = region(geo_0)

print('RMSE модели в регионе №0 = {:.3f}'.format(rmse_0))
print('Средний запас предсказанного сырья в регионе №0 = {:.3f} тыс. баррелей'.format(mean_pred_0)) 
print('Реальный средний запас сырья в регионе №0 = {:.3f} тыс. баррелей'.format(target_valid_0.mean())) 

RMSE модели в регионе №0 = 37.579
Средний запас предсказанного сырья в регионе №0 = 92.593 тыс. баррелей
Реальный средний запас сырья в регионе №0 = 92.079 тыс. баррелей


Регион 1

In [10]:
predictions_1, rmse_1, mean_pred_1, target_valid_1 = region(geo_1)

print('RMSE модели в регионе №0 = {:.3f}'.format(rmse_1))
print('Средний запас предсказанного сырья в регионе №0 = {:.3f} тыс. баррелей'.format(mean_pred_1)) 
print('Реальный средний запас сырья в регионе №0 = {:.3f} тыс. баррелей'.format(target_valid_1.mean())) 

RMSE модели в регионе №0 = 0.893
Средний запас предсказанного сырья в регионе №0 = 68.729 тыс. баррелей
Реальный средний запас сырья в регионе №0 = 68.723 тыс. баррелей


Регион 2

In [11]:
predictions_2, rmse_2, mean_pred_2, target_valid_2 = region(geo_2)

print('RMSE модели в регионе №0 = {:.3f}'.format(rmse_2))
print('Средний запас предсказанного сырья в регионе №0 = {:.3f} тыс. баррелей'.format(mean_pred_2)) 
print('Реальный средний запас сырья в регионе №0 = {:.3f} тыс. баррелей'.format(target_valid_2.mean())) 

RMSE модели в регионе №0 = 40.030
Средний запас предсказанного сырья в регионе №0 = 94.965 тыс. баррелей
Реальный средний запас сырья в регионе №0 = 94.884 тыс. баррелей


Самые богатые регионы по запасам : 0 регион , 2 регион, каждый имеет болеее 90 тыс. баррелей .
    
Все три модели показали близкие к истинным значениям , разница несколько сотых.

RMSE регионов 0 , 2 примерно одинаковый .
    
RMSE региона 1 показал лучший резульат .    
    

## Подготовка к расчёту прибыли

In [12]:
budjet=10000000000 #бюджет 
revenue_per_barrel=450000 #доход за 1 баррель сырья
total_points=500 #точки для исследования
points_for=200 #точки для разработки

In [13]:
clean_budjet = budjet / points_for 
print('Затраты на разработку 1 скважины {:.2f} тыс. баррелей'.format(clean_budjet))#затраты на разработку 1 скважены
product_volume = clean_budjet / revenue_per_barrel #необходимый обьем сырья для безубыточной разработки
print('Необходимых объём сырья для безубыточной разработки новой скважины {:.2f} тыс. баррелей '.format(product_volume))

Затраты на разработку 1 скважины 50000000.00 тыс. баррелей
Необходимых объём сырья для безубыточной разработки новой скважины 111.11 тыс. баррелей 


Средний необходимых объём сырья для безубыточной разработки новой скважины = 111111.11 тыс. баррелей .


Средний необходимый обьем сырья  на разрабоку 1 скважины = 50000000.00 тыс. баррелей .



## Расчёт прибыли и рисков 

In [14]:
def revenue(target, probabilities, count): #функция для расчета прибыли по выбранным предсказаниям 
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return (revenue_per_barrel * selected.sum() - budjet)

In [15]:
print('Прибыль региона №0: {:.2f} млн. руб.'.format(revenue(target_valid_0, pd.Series(predictions_0),200)/10**6))
print('Прибыль региона №1: {:.2f} млн. руб.'.format(revenue(target_valid_1, pd.Series(predictions_1),200)/10**6))
print('Прибыль региона №2: {:.2f} млн. руб.'.format(revenue(target_valid_2, pd.Series(predictions_2),200)/10**6))

Прибыль региона №0: 3320.83 млн. руб.
Прибыль региона №1: 2415.09 млн. руб.
Прибыль региона №2: 2710.35 млн. руб.


In [16]:
state = np.random.RandomState(12345)
def bootstrap(target,probabilities, reg): # функция с bootstrap для поиска распределения прибыли
    values = []
    for i in range(1000): 
        target_subsample = target.sample(500, replace = True, random_state = state)
        probs_subsample = probabilities[target_subsample.index] 
        values.append(revenue(target_subsample, probs_subsample, 200))

    values = pd.Series(values)
    low = values.quantile(0.025) #нижний квантиль
    interval = (values.quantile(0.025), values.quantile(0.975)) #интервал среднего
    mean = values.mean()
    risk = (values < 0).mean() #риск
    
    
    print(reg)
    print('Средняя прибыль:{:.3f} млн. рублей'.format(mean/10**6))
    print('Доверительный интервал среднего:{}'.format(interval))
    print('2.5%-квантиль:{:.3f} млн. рублей'.format(low/10**6))
    print('Риск:{:.2%}'.format(risk))
    print('\n')
    
bootstrap(target_valid_0, pd.Series(predictions_0), 'Регион №0')
bootstrap(target_valid_1, pd.Series(predictions_1), 'Регион №1')
bootstrap(target_valid_2, pd.Series(predictions_2), 'Регион №2')

Регион №0
Средняя прибыль:425.939 млн. рублей
Доверительный интервал среднего:(-102090094.83793654, 947976353.3583689)
2.5%-квантиль:-102.090 млн. рублей
Риск:6.00%


Регион №1
Средняя прибыль:518.259 млн. рублей
Доверительный интервал среднего:(128123231.43308444, 953612982.0669085)
2.5%-квантиль:128.123 млн. рублей
Риск:0.30%


Регион №2
Средняя прибыль:420.194 млн. рублей
Доверительный интервал среднего:(-115852609.16001143, 989629939.8445739)
2.5%-квантиль:-115.853 млн. рублей
Риск:6.20%




## Вывод 

В ходе работы построили модель машинного обучения, которая помогла нам выявить наилучший регион , где добыча принесет большую прибыль.

Проанализировав возможую прибыль и риски техникой Bootstrap , выявили что наименьший риск убытка возникнет в 1 регионе ('geo_1') 0.30% , что подтверждает средний уровень прибыли 518 млн.руб.

На втором месте регион 0('geo_0'), на третьем месте с минимальной средней прибылью регион 2('geo_2')